# To Churn or not to Churn?
## Naive Bayes Classifier
### Author: E. Thompson-Becker
##### Code editied from python weka tutorial recieved from Toronto Metropolitan University, CIND 119 - Introduction to Big Data by Syed Shariyar Murtaza,Ph.D.

Install Weka's python package.

In [1]:
! pip install python-weka-wrapper3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.4 MB 14.7 MB/s 
     |████████████████████████████████| 1.3 MB 11.2 MB/s 
  Created wheel for python-weka-wrapper3: filename=python_weka_wrapper3-0.2.10-py3-none-any.whl size=12993854 sha256=b718ae1206ab6baa894846792d4878e44398e6037f03f810f6b39118ad90d6b5
  Stored in directory: /root/.cache/pip/wheels/a4/e9/93/c8dc5119f22ea38aa2bfbd02c33f4b2a6c6293f1a86283fd91
  Created wheel for python-javabridge: filename=python_javabridge-4.0.3-cp37-cp37m-linux_x86_64.whl size=1628208 sha256=f075f9736772d5e02324f9ae3680eb0ce874653ba2a31bab96bfa2c7fbf1b432
  Stored in directory: /root/.cache/pip/wheels/63/7e/91/01b1bd8d29b4323834feb5cfec49b857fb212e6efc74ce103c
Successfully built python-weka-wrapper3 python-javabridge


Set up java and and launch it in the python environment.

In [2]:
import os
import sys
sys.path
sys.path.append("/usr/lib/jvm/java-11-openjdk-amd64/bin/")
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64/"

import weka.core.jvm as jvm
jvm.start()

DEBUG:weka.core.jvm:Adding bundled jars
DEBUG:weka.core.jvm:Classpath=['/usr/local/lib/python3.7/dist-packages/javabridge/jars/rhino-1.7R4.jar', '/usr/local/lib/python3.7/dist-packages/javabridge/jars/runnablequeue.jar', '/usr/local/lib/python3.7/dist-packages/javabridge/jars/cpython.jar', '/usr/local/lib/python3.7/dist-packages/weka/lib/python-weka-wrapper.jar', '/usr/local/lib/python3.7/dist-packages/weka/lib/weka.jar']
DEBUG:weka.core.jvm:MaxHeapSize=default
DEBUG:weka.core.jvm:Package support disabled


Import data into the python environment.

In [3]:
from google.colab import files
uploaded = files.upload()


Saving churn.arff to churn.arff


Import packages

In [5]:
from weka.core.converters import Loader
from weka.core.classes import Random
from weka.classifiers import Classifier, Evaluation
from weka.filters import Filter

Load data into memory to keep it in the environment. 

In [6]:
loader = Loader(classname="weka.core.converters.ArffLoader")
data_file = "churn.arff"
data = loader.load_file(data_file)

Identify the class index.

In [7]:
class_idx = 20
print("Data will be classified on",data.attribute(class_idx))
data.class_index = class_idx

Data will be classified on @attribute Churn {FALSE,TRUE}


## 1. Naive Bayes classifier including all attributes

Split train and test set by 70/30 split.

In [8]:
train, test = data.train_test_split(70.0, Random(1))

Run Naive Bayes classifier set to use both numeric and categorical data.

In [9]:
nb = Classifier(classname="weka.classifiers.bayes.NaiveBayes")
nb.build_classifier(train)
#understand the NB model by printing it
#print(nb)

Evaluate on the test set.

In [10]:
evl_nb = Evaluation(train)
evl_nb.test_model(nb, test)
print(evl_nb.summary())


Correctly Classified Instances         877               87.7    %
Incorrectly Classified Instances       123               12.3    %
Kappa statistic                          0.4753
Mean absolute error                      0.1971
Root mean squared error                  0.3124
Relative absolute error                 79.5751 %
Root relative squared error             88.9798 %
Total Number of Instances             1000     



Evaluation metrics for both False and True classes. 

In [11]:
#Here are all the metrics for Naive Bayes

print("Classes at different positions are ",data.attribute(class_idx))

print("confusion Matrix")
#Note that the TP here will be for the class at the first position printed by the previous line
print(evl_nb.confusion_matrix)

###############
# Print metrics for the first class: False
##############
class_position=0
print("")
print ("Evaluation from the perspective of class at position "+ str(class_position))
print("TP ",evl_nb.true_positive_rate(class_position))
print("FP",evl_nb.false_positive_rate(class_position))
print("Precision ",evl_nb.precision(class_position))
print("Recall ",evl_nb.recall(class_position))


###############
# Print metrics for the second class: True
##############
class_position=1
print("")
print ("Evaluation from the perspective of class at position "+ str(class_position))
print("TP ",evl_nb.true_positive_rate(class_position))
print("FP",evl_nb.false_positive_rate(class_position))
print("Precision ",evl_nb.precision(class_position))
print("Recall ",evl_nb.recall(class_position))

Classes at different positions are  @attribute Churn {FALSE,TRUE}
confusion Matrix
[[803.  53.]
 [ 70.  74.]]

Evaluation from the perspective of class at position 0
TP  0.9380841121495327
FP 0.4861111111111111
Precision  0.9198167239404352
Recall  0.9380841121495327

Evaluation from the perspective of class at position 1
TP  0.5138888888888888
FP 0.06191588785046729
Precision  0.5826771653543307
Recall  0.5138888888888888


Calculate the F-measure to make it easier to compare the different models. We want to use the model to predict if a customer will churn, so we want to look at the evaluation from the perspective of class at position 1, true. 

In [13]:
#check f-measure to make it easier to compare the models
class_position=1
precision = evl_nb.precision(class_position)
recall = evl_nb.recall(class_position)
F_measure = (2*precision*recall)/(precision+recall)

print('F-measure',F_measure)

F-measure 0.5461254612546126


## 2. Naive Bayes Classification on selected attributes

Use subset of data to select features found in initial analysis. 

In [15]:
#create new data set
sel_data=data.subset(col_range='5,6,8,11,17,18,20,21')

Find the new class index.

In [16]:
class_idx2 = 7
print("Data will be classified on",sel_data.attribute(class_idx2))
sel_data.class_index = class_idx2

Data will be classified on @attribute Churn {FALSE,TRUE}


Create train and test sets using a 70/30 split.

In [17]:
train2, test2 = sel_data.train_test_split(70.0, Random(1))

Make new naive bayes classifier with selected attributes. 

In [18]:
nb2 = Classifier(classname="weka.classifiers.bayes.NaiveBayes")
nb2.build_classifier(train2)
#understand the NB model by printing it
#print(nb)

Evaluate on the test set.

In [19]:
evl_nb2 = Evaluation(train2)
evl_nb2.test_model(nb2, test2)
print(evl_nb2.summary())


Correctly Classified Instances         872               87.2    %
Incorrectly Classified Instances       128               12.8    %
Kappa statistic                          0.3298
Mean absolute error                      0.1862
Root mean squared error                  0.3072
Relative absolute error                 75.1766 %
Root relative squared error             87.5014 %
Total Number of Instances             1000     



Evaluation metrics for the naive bayes classifier.

In [20]:
print("Classes at different positions are ",sel_data.attribute(class_idx2))

print("confusion Matrix")
#Note that the TP here will be for the class at the first position printed by the previous line
print(evl_nb2.confusion_matrix)

###############
# Print metrics for the first class: False
##############
class_position=0
print("")
print ("Evaluation from the perspective of class at position "+ str(class_position))
print("TP ",evl_nb2.true_positive_rate(class_position))
print("FP",evl_nb2.false_positive_rate(class_position))
print("Precision ",evl_nb2.precision(class_position))
print("Recall ",evl_nb2.recall(class_position))


###############
# Print metrics for the second class: True
##############
class_position=1
print("")
print ("Evaluation from the perspective of class at position "+ str(class_position))
print("TP ",evl_nb2.true_positive_rate(class_position))
print("FP",evl_nb2.false_positive_rate(class_position))
print("Precision ",evl_nb2.precision(class_position))
print("Recall ",evl_nb2.recall(class_position))


Classes at different positions are  @attribute Churn {FALSE,TRUE}
confusion Matrix
[[831.  25.]
 [103.  41.]]

Evaluation from the perspective of class at position 0
TP  0.9707943925233645
FP 0.7152777777777778
Precision  0.8897216274089935
Recall  0.9707943925233645

Evaluation from the perspective of class at position 1
TP  0.2847222222222222
FP 0.029205607476635514
Precision  0.6212121212121212
Recall  0.2847222222222222


In [21]:
#check f-measure to make it easier to compare the models
class_position=1
precision = evl_nb2.precision(class_position)
recall = evl_nb2.recall(class_position)
F_measure = (2*precision*recall)/(precision+recall)

print('F-measure',F_measure)

F-measure 0.3904761904761905


In [ ]:
#stop the JVM (Java Virtual Machine)
jvm.stop()